In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
#tf.enable_eager_execution()

In [3]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import DataSet
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
import os
import six
from collections import namedtuple
from tqdm import tqdm

In [5]:
df = pd.read_csv('Data/p7_master_window.csv')
df.fillna(value=0, inplace=True)
df = df[['op_num_people', 'of_pose_distance', 'of_gaze_distance', 'of_success', 'of_confidence', 'ros_difficulty', 'ros_mistakes_session', 'ros_ts_robot_talked', 'engagement']]

len_df = len(df)
train_split_ratio = 0.6
val_split_ratio = 0.1
test_split_ratio = 0.3
train_split = int(train_split_ratio * len_df)
val_split = int(val_split_ratio * len_df)
test_split = int(test_split_ratio * len_df)

train_df, val_df, test_df = df.iloc[:train_split,:], df.iloc[train_split:train_split + val_split,:]\
                            ,df.iloc[train_split + val_split:,:]

feats = []
for i in df.columns:
    if i!= 'engagement':
        feats.append(i)
label = ['engagement']

X = df[feats]
Y = df[label]
X_train = train_df[feats]
y_train = train_df[label]#.astype(int)
X_test = test_df[feats]
y_test = test_df[label]#.astype(int)
X_vals = val_df[feats]
y_vals = val_df[label]#.astype(int)

In [6]:
X_pos = train_df[train_df['engagement'] == 1][feats]
y_pos = train_df[train_df['engagement'] == 1][label].astype(int)
X_neg = train_df[train_df['engagement'] == 0][feats]
y_neg = train_df[train_df['engagement'] == 0][label].astype(int)

In [7]:
print(X_train.shape, y_train.shape)
print(X_vals.shape, y_vals.shape)
print(X_test.shape, y_test.shape)

(12075, 8) (12075, 1)
(2012, 8) (2012, 1)
(6039, 8) (6039, 1)


In [8]:
print(y_train.engagement.value_counts())
print(y_vals.engagement.value_counts())
print(y_test.engagement.value_counts())

1.0    7436
0.0    4639
Name: engagement, dtype: int64
1.0    1109
0.0     903
Name: engagement, dtype: int64
1.0    3484
0.0    2555
Name: engagement, dtype: int64


In [9]:
# Ratio of positive examples in training
y_train.engagement.value_counts()[1] / (y_train.engagement.value_counts()[0]
                                       + y_train.engagement.value_counts()[1])

0.6158178053830228

In [10]:
# Constants
LR = 0.001
NUM_STEPS = 1000
seed = 42

In [11]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
tf.logging.set_verbosity(tf.logging.ERROR)

In [12]:
import sys
sys.argv = " --exp ours".split(" ")

In [13]:
flags = tf.flags
flags.DEFINE_float('pos_ratio', 0.6158178053830228, 'Ratio of positive examples in training')
flags.DEFINE_integer('nrun', 1, 'Number of runs')
flags.DEFINE_integer('ntest', 6039, 'Number of testing examples')
flags.DEFINE_integer('ntrain', 12075, 'Number of training examples')
flags.DEFINE_integer('nval', 2012, 'Number of validation examples')
flags.DEFINE_bool('verbose', True, 'Whether to print training progress')
flags.DEFINE_bool('tensorboard', False, 'Whether to save training progress')
flags.DEFINE_string('exp', 'ours', 'Which experiment to run')
FLAGS = tf.flags.FLAGS


In [14]:
Config = namedtuple('Config', [
    'reweight', 'lr', 'num_steps', 'random', 'ratio_weighted', 'nval', 'hard_mining', 'bsize'
])

exp_repo = dict()

def RegisterExp(name):
    def _decorator(f):
        exp_repo[name] = f
        return f

    return _decorator

@RegisterExp('ours')
def ours_config():
    return Config(
        reweight=True,
        num_steps=NUM_STEPS,
        lr=LR,
        random=False,
        ratio_weighted=False,
        hard_mining=False,
        bsize=100,
        nval=FLAGS.nval)

In [17]:
train_set = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
val_set = tf.data.Dataset.from_tensor_slices((X_vals.values, y_vals.values))
test_set = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))

train_pos_set = tf.data.Dataset.from_tensor_slices((X_pos.values, y_pos.values))
train_neg_set = tf.data.Dataset.from_tensor_slices((X_neg.values, y_neg.values))

In [18]:
pos_ratio = FLAGS.pos_ratio
ntrain = FLAGS.ntrain
nval = FLAGS.nval
ntest = FLAGS.ntest
seed = 42
exp_name = 'ours'
config = exp_repo['ours']()
verbose = True

In [20]:
feature_count = len(feats)
label_count = 1
hidden_layers = feature_count - 1
is_training=tf.Variable(True,dtype=tf.bool)

In [21]:
def get_acc(logits, y):
    prediction = tf.cast(tf.sigmoid(logits) > 0.5, tf.float32)
    return tf.reduce_mean(tf.cast(tf.equal(prediction, y), tf.float32))

def get_model(inputs, labels, is_training=True, dtype=tf.float32, w_dict=None, ex_wts=None, reuse=None):
    
    w_dict = {} 
    
#     def _get_var(name, shape, dtype, initializer):
#         key = tf.get_variable_scope().name + '/' + name
#         if key in w_dict:
#             return w_dict[key]
#         else:
#             var = tf.get_variable(name, shape, dtype, initializer=initializer)
#             w_dict[key] = var
#             return var
    
    with tf.variable_scope('Model', reuse=reuse):
        inputs_ = tf.cast(inputs, dtype)
        labels = tf.cast(labels, dtype)
        act = tf.nn.relu
        initializer = tf.contrib.layers.xavier_initializer()

        h0 = tf.layers.dense(inputs, hidden_layers, activation=act, kernel_initializer=initializer)
        h1 = tf.layers.dense(h0, hidden_layers, activation=act, kernel_initializer=initializer)
        h2 = tf.layers.dense(h1, hidden_layers, activation=act, kernel_initializer=initializer)
        h3 = tf.layers.dense(h2, hidden_layers, activation=act, kernel_initializer=initializer)
        h4 = tf.layers.dense(h3, label_count, activation=None)   

        logits = h4

        if ex_wts is None:
            # Average loss.
            loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))
        else:
            # Weighted loss.
            loss = tf.reduce_sum(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels) * ex_wts)
    return w_dict, loss, logits

In [22]:
def evaluate(sess, x_, y_, acc_, train_set, test_set):
    
    train_set_ie = train_set.batch(100)
    train_iterator_ie = train_set_ie.make_one_shot_iterator()

    test_set_ie = test_set.batch(100)
    test_iterator_ie = test_set_ie.make_one_shot_iterator()
    
    # Calculate final results.
    acc_sum = 0.0
    acc_test_sum = 0.0
    train_bsize = 100
    for step in six.moves.xrange(5000 // train_bsize):
        x, y = train_iterator_ie.get_next()
        acc = sess.run(acc_, feed_dict={x_: x.eval(), y_: y.eval()})
        acc_sum += acc

    test_bsize = 100
    for step in six.moves.xrange(500 // test_bsize):
        x_test, y_test = test_iterator_ie.get_next()
        acc = sess.run(acc_, feed_dict={x_: x_test.eval(), y_: y_test.eval()})
        acc_test_sum += acc

    train_acc = acc_sum / float(5000 // train_bsize)
    test_acc = acc_test_sum / float(500 // test_bsize)
    return train_acc, test_acc

In [23]:
def reweight_autodiff(inp_a,
                      label_a,
                      inp_b,
                      label_b,
                      bsize_a,
                      bsize_b,
                      eps=0.0,
                      gate_gradients=1):
    """Reweight examples using automatic differentiation.
    :param inp_a:             [Tensor]    Inputs for the noisy pass.
    :param label_a:           [Tensor]    Labels for the noisy pass.
    :param inp_b:             [Tensor]    Inputs for the clean pass.
    :param label_b:           [Tensor]    Labels for the clean pass.
    :param bsize_a:           [int]       Batch size for the noisy pass.
    :param bsize_b:           [int]       Batch size for the clean pass.
    :param eps:               [float]     Minimum example weights, default 0.0.
    :param gate_gradients:    [int]       Tensorflow gate gradients, reduce concurrency.
    """
    ex_wts_a = tf.zeros([bsize_a], dtype=tf.float32)
    ex_wts_b = tf.ones([bsize_b], dtype=tf.float32) / float(bsize_b)
    
    w_dict, loss_a, logits_a = get_model(
        inp_a, label_a, ex_wts=ex_wts_a, is_training=True, reuse=True)
    
    var_names = w_dict.keys()
    var_list = [w_dict[kk] for kk in var_names]
    
    grads = tf.gradients(loss_a, var_list, gate_gradients=gate_gradients)

    var_list_new = [vv - gg for gg, vv in zip(grads, var_list)]
    w_dict_new = dict(zip(var_names, var_list_new))
    _, loss_b, logits_b = get_model(
        inp_b, label_b, ex_wts=ex_wts_b, is_training=True, reuse=True, w_dict=w_dict_new)
    
    grads_ex_wts = tf.gradients(loss_b, [ex_wts_a], gate_gradients=gate_gradients)[0]
    ex_weight = -grads_ex_wts
    ex_weight_plus = tf.maximum(ex_weight, eps)
    ex_weight_sum = tf.reduce_sum(ex_weight_plus)
    ex_weight_sum += tf.to_float(tf.equal(ex_weight_sum, 0.0))
    ex_weight_norm = ex_weight_plus / ex_weight_sum
    return ex_weight_norm

In [24]:
def reweight_random(bsize, eps=0.0):
    """Reweight examples using random numbers.
    
    :param bsize:             [int]       Batch size.
    :param eps:               [float]     Minimum example weights, default 0.0.
    """
    ex_weight = tf.random_normal([bsize], mean=0.0, stddev=1.0)
    ex_weight_plus = tf.maximum(ex_weight, eps)
    ex_weight_sum = tf.reduce_sum(ex_weight_plus)
    ex_weight_sum += tf.to_float(tf.equal(ex_weight_sum, 0.0))
    ex_weight_norm = ex_weight_plus / ex_weight_sum
    return ex_weight_norm

In [25]:
bsize = config.bsize
num_steps = config.num_steps

In [27]:
with tf.Graph().as_default(), tf.Session() as sess:
    train_set_i = train_set.repeat(num_steps).batch(bsize)
    train_iterator = train_set_i.make_one_shot_iterator()
    val_set_i = val_set.repeat(num_steps).batch(bsize)
    val_iterator = val_set_i.make_one_shot_iterator()
    
    config = exp_repo['ours']()
    bsize = config.bsize
    
    x_ = tf.placeholder(tf.float32,[None,feature_count], name='x')
    y_ = tf.placeholder(tf.float32, [None,label_count], name='y')
    
    x_val_ = tf.placeholder(tf.float32, [None, feature_count], name='x_val')
    y_val_ = tf.placeholder(tf.float32, [None,label_count], name='y_val')
    ex_wts_ = tf.placeholder(tf.float32, [None], name='ex_wts')
    lr_ = tf.placeholder(tf.float32, [], name='lr')

    # Build training model.
    with tf.name_scope('Train'):
        _, loss_c, logits_c = get_model(
            x_, y_, is_training=True, dtype=tf.float32, w_dict=None, ex_wts=ex_wts_, reuse=None)
        train_op = tf.train.MomentumOptimizer(config.lr, 0.9).minimize(loss_c)
        
    # Build evaluation model.
    with tf.name_scope('Val'):
        _, loss_eval, logits_eval = get_model(
            x_,
            y_,
            is_training=False,
            dtype=tf.float32,
            w_dict=None,
            ex_wts=ex_wts_,
            reuse=True)
        acc_ = get_acc(logits_eval, y_)
    
    # Build reweighting model.
    if config.reweight:
        ex_weights_ = reweight_random(bsize)
#         ex_weights_ = reweight_autodiff(
#             x_,
#             y_,
#             x_val_,
#             y_val_,
#             bsize,
#             min(bsize, nval),
#             eps=0.0,
#             gate_gradients=1)    
        
    exp_logger = None

    lr = config.lr
    num_steps = config.num_steps

    acc_sum = 0.0
    acc_test_sum = 0.0
    loss_sum = 0.0
    count = 0
    sess.run(tf.global_variables_initializer())
    for step in six.moves.xrange(num_steps):
        x, y = train_iterator.get_next()
        x_val, y_val = val_iterator.get_next()

        # Use 50% learning rate for the second half of training.
        if step > num_steps // 2:
            lr = config.lr / 2.0
        else:
            lr = config.lr

        ex_weights = sess.run(
            ex_weights_, feed_dict={x_: x.eval(),
                                    y_: y.eval(),
                                    x_val_: x_val.eval(),
                                    y_val_: y_val.eval()})
        loss, acc, _ = sess.run(
            [loss_c, acc_, train_op],
            feed_dict={
                x_: x.eval(),
                y_: y.eval(),
                x_val_: x_val.eval(),
                y_val_: y_val.eval(),
                ex_wts_: ex_weights,
                lr_: lr
            })
        if (step + 1) % 100 == 0:
            train_acc, test_acc = evaluate(sess, x_, y_, acc_, train_set, test_set)
            if verbose:
                print('Step', step + 1, 'Loss', loss, 'Train acc', train_acc, 'Test acc',
                      test_acc)
            if FLAGS.tensorboard:
                exp_logger.log(step + 1, 'train acc', train_acc)
                exp_logger.log(step + 1, 'test acc', test_acc)
                exp_logger.flush()
            acc_sum = 0.0
            loss_sum = 0.0
            acc_test_sum = 0.0
            count = 0

    # Final evaluation.
    train_acc, test_acc = evaluate(sess, x_, y_, acc_, train_set, test_set)
    if verbose:
        print('Final', 'Train acc', train_acc, 'Test acc', test_acc)

/home/bala/anaconda2/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/data/ops/iterator_ops.py:408: UserWarning: An unusually high number of `Iterator.get_next()` calls was detected. This often indicates that `Iterator.get_next()` is being called inside a training loop, which will cause gradual slowdown and eventual resource exhaustion. If this is the case, restructure your code to call `next_element = iterator.get_next()` once outside the loop, and use `next_element` as the input to some computation that is invoked inside the loop.
  warnings.warn(GET_NEXT_CALL_WARNING_MESSAGE)


Step 100 Loss 121.59438 Train acc 0.6020000010728837 Test acc 0.5060000002384186
Step 200 Loss 89.85155 Train acc 0.39799999933689834 Test acc 0.4939999938011169
Step 300 Loss 64.72139 Train acc 0.6020000010728837 Test acc 0.5060000002384186
Step 400 Loss 133.56561 Train acc 0.6020000010728837 Test acc 0.5060000002384186
Step 500 Loss 97.673965 Train acc 0.39799999933689834 Test acc 0.4939999938011169
Step 600 Loss 48.5645 Train acc 0.6020000010728837 Test acc 0.5060000002384186
Step 700 Loss 160.89932 Train acc 0.6020000010728837 Test acc 0.5060000002384186
Step 800 Loss 73.17954 Train acc 0.39799999933689834 Test acc 0.4939999938011169
Step 900 Loss 67.498695 Train acc 0.6020000010728837 Test acc 0.5060000002384186
Step 1000 Loss 52.861008 Train acc 0.6020000010728837 Test acc 0.5060000002384186
Final Train acc 0.6020000010728837 Test acc 0.5060000002384186
